In [7]:
import cv2
import numpy as np
import os
import subprocess
import torch
import torch.nn as nn

In [6]:
# Set the URLs for the YOLO files
yolo_config_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
yolo_weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
yolo_labels_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'

# Set the paths to save the YOLO files
yolo_config  = 'yolo_files/yolo.cfg'
yolo_weights = 'yolo_files/yolo.weights'
yolo_labels  = 'yolo_files/yolo.names'

# # Download the YOLO files if they do not exist
# if not os.path.exists(yolo_config):
#     subprocess.run(['curl', '-o', yolo_config, yolo_config_url])
# if not os.path.exists(yolo_weights):
#     subprocess.run(['curl', '-o', yolo_weights, yolo_weights_url])
# if not os.path.exists(yolo_labels):
#     subprocess.run(['curl', '-o', yolo_labels, yolo_labels_url])


In [16]:
# Load YOLO model
net = cv2.dnn.readNetFromDarknet(yolo_config, yolo_weights)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels
with open(yolo_labels, 'r') as f:
    classes = [line.strip() for line in f.readlines()]


IndexError: invalid index to scalar variable.

In [ ]:
# Path to your dataset
dataset_path = 'yolo_images/Face_Data/Face_Dataset/0'

# Loop over images in the dataset folder
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        image_path = os.path.join(root, file)
        image = cv2.imread(image_path)

        # Perform face detection
        blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Process the detections
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # If the detected class is "face" and confidence is high enough
                if classes[class_id] == 'face' and confidence > 0.5:
                    # Extract the bounding box coordinates
                    center_x = int(detection[0] * image.shape[1])
                    center_y = int(detection[1] * image.shape[0])
                    width = int(detection[2] * image.shape[1])
                    height = int(detection[3] * image.shape[0])

                    # Draw the bounding box on the image
                    x = int(center_x - width / 2)
                    y = int(center_y - height / 2)
                    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)

        # Display the image with detected faces
        cv2.imshow('Face Detection', image)
        cv2.waitKey(0)

cv2.destroyAllWindows()